This example notebook uses synthetic data produced by PZFlow in combination with several predefined SED templates and filter definition files.

In [ ]:
from rail.estimation.algos.lephare import LephareInformer, LephareEstimator
import numpy as np
import lephare as lp
from rail.core.stage import RailStage
from rail.core.data import TableHandle
import matplotlib.pyplot as plt

DS = RailStage.data_store
DS.__class__.allow_overwrite = True

Here we load previously created synthetic data

In [ ]:
import tables_io

trainFile = './data/output_table_conv_train.hdf5'
testFile = './data/output_table_conv_test.hdf5'

traindata_io = DS.read_file("training_data", TableHandle, trainFile)
testdata_io = DS.read_file("test_data", TableHandle, testFile)

In [ ]:
traindata_io

Retrieve all the required filter and template files

In [ ]:
lephare_config_file = "../tests/data/lsst.para"
lephare_config = lp.read_config(lephare_config_file)

lp.data_retrieval.get_auxiliary_data(keymap=lephare_config)

We use the inform stage to create the library of SEDs with various redshifts, extinction parameters, and reddening values. This typically takes ~3-4 minutes.

In [ ]:
inform_lephare = LephareInformer.make_stage(
    name="inform_Lephare",
    nondetect_val=np.nan,
    model="lephare.pkl",
    hdf5_groupname="",
    # Use a sparse redshift grid to speed up the notebook
    lephare_config=lp.keymap_to_string_dict(lephare_config),
    zmin=0,
    zmax=5,
    nzbins=51,
)

inform_lephare.inform(traindata_io)

Now we take the sythetic test data, and find the best fits from the library. This results in a PDF, zmode, and zmean for each input test data. Takes ~2 minutes to run on 1500 inputs.

In [ ]:
estimate_lephare = LephareEstimator.make_stage(
    name="test_Lephare",
    nondetect_val=np.nan,
    model=inform_lephare.get_handle("model"),
    hdf5_groupname="",
    aliases=dict(input="test_data", output="lephare_estim"),
)

lephare_estimated = estimate_lephare.estimate(testdata_io)

An example lephare PDF and comparison to the true value

In [ ]:
indx = 0
zgrid = np.linspace(0,3,300)
plt.plot(zgrid, np.squeeze(lephare_estimated.data[indx].pdf(zgrid)), label='Estimated PDF')
plt.axvline(x=testdata_io.data['redshift'][indx], color='r', label='True redshift')
plt.legend()
plt.xlabel('z')
plt.show()

More example fits

In [ ]:
indxs = [8, 16, 32, 64, 128, 256, 512, 1024]
zgrid = np.linspace(0,3,300)
fig, axs = plt.subplots(2,4, figsize=(20,6))
for i, indx in enumerate(indxs):
    ax = axs[i//4, i%4]
    ax.plot(zgrid, np.squeeze(lephare_estimated.data[indx].pdf(zgrid)), label='Estimated PDF')
    ax.axvline(x=testdata_io.data['redshift'][indx], color='r', label='True redshift')
    ax.set_xlabel('z')

Histogram of the absolute difference between lephare estimate and true redshift

In [ ]:
estimate_diff_from_truth = np.abs(lephare_estimated.data.ancil['zmode'] - testdata_io.data['redshift'])

plt.figure()
plt.hist(estimate_diff_from_truth, 100)
plt.xlabel('abs(z_estimated - z_true)')
plt.show()